# ML-CodeParrot Pretraining Process
- BigQuery에서 제공하는 Github Dataset 중 ML Library(torch, transformers, ..., sklearn)를 사용하는 코드만 추출하여 Pretraining <br>
- Pretrained Tokenizer, Model은 모두 Huggingface Hub에서 [load](https://huggingface.co/rockmiin)하여 사용 가능
- [기존 CodeParrot모델](transformersbook/codeparrot-small)과 결과를 비교하며 진행할 예정
## 전체 프로세스
- ### Extract ML-Github Dataset
- ### Tokenizer
- ### Model
- ### Experiment
- ### Conclusion

## Extract ML-Github Dataset
- BigQuery에서 하위 Query구문을 이용하여 `torch`, `sklearn`, `huggingface library(transformers, datasets, tokenizers)`를 사용하는 py파일만 추출 (2분 내로 처리)<br>
- 총 2.7TB에서 5.61GB(446595 samples)를 추출하여 사용 ([CodeParrot](transformersbook/codeparrot-small) 모델에 비해 3%에 해당하는 데이터셋 사용)
- 추출된 데이터를 9:1 비율로 [Train](https://huggingface.co/datasets/rockmiin/ml-codeparrot-train), [Valid](https://huggingface.co/datasets/rockmiin/ml-codeparrot-valid) Dataset을 나누어 Huggingface Hub에 업로드. 


In [ ]:
SELECT
  f.repo_name, f.path, c.copies, c.size, c.content, l.license
FROM
  `bigquery-public-data.github_repos.files` AS f
JOIN
  `bigquery-public-data.github_repos.contents` AS c
ON
  f.id = c.id
JOIN
  `bigquery-public-data.github_repos.licenses` AS l
ON
  f.repo_name = l.repo_name
WHERE
  NOT c.binary
  AND ((f.path LIKE '%.py')
    AND (c.size BETWEEN 1024 AND 1048575))
  AND REGEXP_CONTAINS(c.content, r'torch|sklearn|transformers|datasets|tokenizers')

## Tokenizer
- 사전학습된 codeparrot과 ml-codeparrot tokenizer의 topk token list 확인
- 한 쪽 tokenizer vocab에만 포함되어 있는 token 확인 

In [149]:
from transformers import AutoTokenizer

ml_tokenizer= AutoTokenizer.from_pretrained('rockmiin/ml-codeparrot')
org_tokenizer= AutoTokenizer.from_pretrained('transformersbook/codeparrot-small')

In [151]:
topk= 1000
top_ml_vocab= [tok for tok, idx in sorted(ml_tokenizer.vocab.items(), key= lambda x: x[1])[257:257+topk]]
top_org_vocab= [tok for tok, idx in sorted(org_tokenizer.vocab.items(), key= lambda x: x[1])[257:257+topk]]

In [153]:
top_ml_vocab[:10]

['ĠĠ', 'ĠĠĠĠ', 'ĠĠĠ', 'ĠĠĠĠĠĠĠĠ', 'in', 'se', 'at', 're', 'ĠĠĠĠĠĠĠ', 'or']

In [154]:
top_org_vocab[:10]

['ĠĠ', 'ĠĠĠĠ', 'ĠĠĠ', 'ĠĠĠĠĠĠĠĠ', 'se', 'in', 'ĠĠĠĠĠĠĠ', 're', 'on', 'te']

In [20]:
# ml-vocab topk에는 있지만 org-vocab에 존재하지 않는 token
print([tok for tok in top_ml_vocab if tok not in top_org_vocab])
# [train, iter, shape, features]

['ion', 'ha', 'las', 'lass', 'igh', 'ĠX', 'atu', 'odel', 'rain', 'hape', 'eigh', 'amples', 'ath', 'arg', 'atures', 'fit', 'equal', 'var', 'learn', 'samples', 'iter', 'ob', 'train', 'shape', 'ive', 'imator', 'np', 'ial', 'Ġnum', 'ameter', 'label', 'ues', 'sor', 'features', 'raise', 'ib', 'ifi', 'Ġarray', 'lot', 'ansform', 'weigh', 'Ġmodel', 'ents', 'ast', 'estimator', 'iel', 'Ġlabel', '[:', 'vari', 'score', 'Ġsk', 'pend', 'tric', 'Ġrandom', 'Val', 'ies', 'Ġtr', 'ef', 'tt', 'Ġdataset', 'random', 'nel', 'Ġsklearn', 'rix', 'atrix', 'gres', 'Ġsc', 'Ġfe', 'umn', 'ature', '================', 'Ġtrain', 'Ġ>>', 'pha', 'Ġ>>>', 'ord', 'ameters', 'weight', 'ifier', 'Ġshape', 'Ġax', 'ice', 'Ġinput', 'ors', 'pect', 'Ġclf', 'sets', 'lasses', 'dataset', 'Ġob', 'Ġmax', 'plit', 'au', 'ices', 'ations', 'ttp', 'Ġnumber', 'most', 'Ġj', 'plot', 'ValueError', 'ision', 'ensor', 'ĊĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠ', 'ition', 'Ġpl', 'zer', 'ep', 'ils', 'num', 'ernel', 'Ġwhe', 'oin', 'onents', 'almost', 'u

In [21]:
# ml-vocab topk에는 있지만 org-vocab에 존재하지 않는 token
print([tok for tok in top_org_vocab if tok not in top_ml_vocab])
# [url, db, jango, assertEqual]

['ti', 'tion', 'fi', 'si', 'xt', 'alue', 'la', '::', 'eld', 'gs', 'bu', 'bj', 'lin', 'ls', 'ht', 'bject', 'ci', 'tem', 'our', 'app', 'module', 'wor', 'mm', 'tri', 'Ġar', 'lic', 'Ġma', 'ource', 'assertEqual', 'mode', 'url', 'ry', "'),", 'Ġra', 'fa', 'ader', 'ĊĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠ', 'sp', 'tp', 'Ġpa', 'ca', 'db', 'field', 'Ġns', 'ssage', 'ĠĊĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠĠ', 'vice', 'ten', 'scri', 'ponse', "'):", 'EN', 'jo', 'tes', 'AL', 'late', 'user', 'ress', 'wa', 'ari', 'tic', 'method', 'ock', 'tent', 'ET', 'vent', 'gn', 'object', 'tribu', "Ġ['", 'Ġ}', 'Field', 'work', 'pp', 'Type', 'ze', 'temp', 'ception', 'stri', 'lay', 'Ġmode', 'ec', 'kw', 'ble', 'son', 'ST', 'Ġcls', '11', 'net', 'jang', 'tions', 'lif', '.__', 'tive', 'jango', 'Ġmodule', 'andle', 'mmand', 'Ġtry', 'lient', '////', 'Ġexcept', 'De', 'update', 'string', 'ters', 'AN', 'Ġro', 'umber', "''", 'atus', 'SE', 'Ġfield', 'gin', 'LE', 'RE', 'peci', 'ght', 'Pro', 'Ex', 'Ġuser', 'group', 'ape', 'Ġ##', 'opy', 'Ġpath', 'node', 'UT', 'uti', "'}", '

- 학습에 사용된 데이터에 따라 vocab 구성이 꽤 많이 바뀌는 것을 확인 

## Model
- [huggingface codeparrot repository](https://github.com/huggingface/transformers/tree/main/examples/research_projects/codeparrot) 코드 사용
- [ml-codeparrot-train dataset](https://huggingface.co/datasets/rockmiin/ml-codeparrot-train)을 **2epoch**(200,000step) 학습
- baseline model로 [gpt2](https://huggingface.co/gpt2)를 사용
- A5000 24G 1GPU를 사용(약 28시간 소요)

###Dataset
The entire data was divided into 9:1 and divided into train and valid dataset.
| Dataset                | Raw size |
|----------------------|----------------|
| ml-codeparrot-train            | 5.05GB            |
| ml-codeparrot-valid            | 0.56GB          |

### Baseline Models
Pretraining was performed using the gpt2 
| Model                | Model size | Vocab size |
|----------------------|----------------|-------------|
| gpt2            | 117M            | 32768         |

### Monitoring
**Train Loss**<br>
<center><img src="./images/train_loss.png" width="900" height="300"></center>

**Eval Loss**<br>
<center><img src="./images/eval_loss.png" width="900" height="300"></center>

**Eval Perplexity**<br>
<center><img src="./images/eval_perplexity.png" width="900" height="300"></center>

## Experiment
- ml-codeparrot과 codeparrot의 generation 결과 비교

In [2]:
from transformers import pipeline, set_seed

model_ckpt= 'rockmiin/ml-codeparrot'
generation = pipeline('text-generation', model=model_ckpt)

/home/deepai/.pyenv/versions/anaconda3-5.3.1/envs/copilot/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
org_model_ckpt= 'transformersbook/codeparrot-small'
org_generation = pipeline('text-generation', model=org_model_ckpt)

In [14]:
import re

def first_block(string):
    return re.split('\nclass|\ndef|\n#|\n@|\nprint|\nif', string)[0].rstrip()

def complete_code(pipe, prompt, max_length=64, num_completions=4, seed=42):
    set_seed(seed)
    gen_kwargs = {"temperature":0.6, "top_p":0.90, "top_k":0, "num_beams":1,
                  "do_sample":True,}
    code_gens = pipe(prompt, num_return_sequences=num_completions, 
                            max_length=max_length, **gen_kwargs)
    code_strings = []
    for code_gen in code_gens:
        generated_code = first_block(code_gen['generated_text'][len(prompt):])
        code_strings.append(generated_code)
    print(('\n'+'='*80 + '\n').join(code_strings))

In [15]:
prompt = '''
def encode(sentence, tokenizer):
    """
    Return tokenized list of input sentences.
    
    Example:
    sentence: ["Hi", "how are you"] -> output: [[1], [34, 5656, 32]]
    """'''

complete_code(generation, prompt, max_length=64)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



    return [
        tokenizer(sentence) for sentence in sentence]

    return [x.encode(tokenizer.tokenize(x)) for x in

    tokenized = tokenizer.encode(sentence)
    tokenized = tokenized.encode(

    return [
        [tokenizer(sentence, tokenizer, max_tokens=max


In [16]:
prompt = '''
def encode(sentence, tokenizer):
    """
    Return tokenized list of input sentences.
    
    Example:
    sentence: ["Hi", "how are you"] -> output: [[1], [34, 5656, 32]]
    """'''

complete_code(org_generation, prompt, max_length=64)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



    # TODO: use tokenize.tokenize to handle sentence
    sentence = tokenizer

    return ''.join(sentence.split())

    # TODO: Use tokenizer to decode sentence
    sentence = tokenizer.tokenize

    return [encode_sentence(sentence, tokenizer) for sentence in sentence


In [26]:
prompt = '''
import torch
def concat_tensor(a, b):
    """
    Return concatenated tensor of two input tensors.
    Assume the sizes of two tensors are equal.
    """'''

complete_code(generation, prompt, max_length=64)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



    return [
        torch.zeros(1, len(a), dtype=torch.float),
        torch.zeros(1, len

    return a.to(b)

    return torch.cat(a, dim=-1)

    return torch.stack([a, b], axis=-1)


In [30]:
prompt = '''
import torch
def concat_tensor(a, b):
    """
    Return concatenated tensor of two input tensors.
    Assume the sizes of two tensors are equal.
    """'''

complete_code(org_generation, prompt, max_length=64)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



    return a.concat(b)

    return a * b

    # Compute the concatenated tensors
    if isinstance(a, tensor.Tensor):
        a = a.value()
    # Add the concatenated

    return torch.cat(a, b)
